In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as skmetrics


In [ ]:
# def fit_data(data1, y_var='pearsons_size', x_var='cortical_distance_label'):
#     ydata = data1[y_var].copy()
#     xdata0 = data1[x_var].copy()
#     xdata0 = xdata0[:, np.newaxis]
#     xdata = StandardScaler().fit_transform(xdata0) #xd = scaler.fit_transform(xdata)
#     xdata_scaled = pd.DataFrame(np.squeeze(xdata), columns=[x_var])
#     xdata_scaled.dropna().head()
#     # Split data
#     x_train, x_test, y_train, y_test = train_test_split(xdata_scaled, 
#                                         ydata, test_size = 0.2, random_state = 42)
#     LR = LinearRegression().fit(x_train,y_train)
#     # predicting the accuracy score
#     y_prediction =  LR.predict(x_test)
#     R2=skmetrics.r2_score(y_test,y_prediction)
#     MSE = skmetrics.mean_squared_error(y_test,y_prediction)
#     print('r2 score: %.2f' % R2)
#     print('mean_sqrd_error: %.2f' % MSE)
#     print('RMSE: %.2f' % np.sqrt(MSE))
#     residuals = y_test.values - y_prediction
#     variance_resid = np.var(residuals)
#     variance_test = np.var(y_test)
#     variance_explained = 1-(variance_resid/variance_test)
#     print(variance_explained)

#     return LR, x_test, y_test

def fit_data(data1, y_var='pearsons_size', x_var='cortical_distance_label'):
    ydata = data1[y_var].copy()
    xdata0 = data1[x_var].copy()
    xdata0 = xdata0[:, np.newaxis]
    xdata = StandardScaler().fit_transform(xdata0) #xd = scaler.fit_transform(xdata)
    xdata_scaled = pd.DataFrame(np.squeeze(xdata), columns=[x_var])
    xdata_scaled.dropna().head()
    # Split data
    LR = LinearRegression().fit(xdata_scaled,ydata)
    # predicting the accuracy score
    y_prediction =  LR.predict(xdata_scaled)
    R2=skmetrics.r2_score(ydata,y_prediction)
    MSE = skmetrics.mean_squared_error(ydata,y_prediction)
    print('r2 score: %.2f' % R2)
    print('mean_sqrd_error: %.2f' % MSE)
    print('RMSE: %.2f' % np.sqrt(MSE))
    residuals = ydata.values - y_prediction
    variance_resid = np.var(residuals)
    variance_test = np.var(ydata)
    variance_explained = 1-(variance_resid/variance_test)
    print(variance_explained)

    return LR, xdata_scaled, ydata


In [ ]:

ctx_maxdist=500
df, dist_lut = cr.get_bins_and_cut(DISTS[DISTS.cortical_distance<=ctx_maxdist], 
                                   ctx_step=50, overlap_step=0.05,
                                   equal_bins=False)
va = 'Li'
vg = df[(df.visual_area==va)].copy().dropna()
print(vg.cortical_distance.max())
factors =['%s_label' % x for x in dist_vars]
print(factors)

In [ ]:
var1, var2 = factors

data1 = vg.groupby([var1, 'datakey']).median().reset_index()
data2 = vg.groupby([var2, 'datakey']).median().reset_index()

In [ ]:
scatter_kws={'s': 1, 'alpha': 0.5, 'color': 'k'}
fig, axn = pl.subplots(1, 2, figsize=(6,3))

ax=axn[0]
sns.regplot(x=var1, y='pearsons_size', data=data1, ax=ax,
            scatter_kws=scatter_kws)
ax=axn[1]
sns.regplot(x=var2, y='pearsons_size', data=data2, ax=ax,
            scatter_kws=scatter_kws)
pl.subplots_adjust(bottom=0.2, right=0.8)


In [ ]:
fig, ax = pl.subplots()
sns.regplot(vg[var1].values, vg[var2].values, ax=ax)
ax.set_box_aspect(1)
pl.subplots_adjust(bottom=0.2, right=0.9)
print(spstats.pearsonr(vg[var1].values, vg[var2].values))


In [ ]:
# Fit to cortical dist (var1)
LR1, xdata1, ydata1 = fit_data(data1, y_var='pearsons_size', x_var=var1)
y_prediction1 = LR.predict(xdata1)
# Fit to RF overlap (var2)
LR2, xdata2, ydata2 = fit_data(data2, y_var='pearsons_size', x_var=var2)
y_prediction2 = LR2.predict(x_test2)

fig, axn = pl.subplots(1,3, figsize=(6,3))
ax=axn[0]
ax.plot(xdata1, ydata1, color='k',marker='.', lw=0)
ax.plot(xdata1, y_prediction1, color='r',marker='.', lw=0)
ax.set_title(var1)
ax.set_box_aspect(1)
ax=axn[1]
ax.plot(x_test2, y_test2, color='k',marker='.', lw=0)
ax.plot(x_test2, y_prediction2, color='r',marker='.', lw=0)
ax.set_title(var2)
ax.set_box_aspect(1)

ax=axn[2]
residuals2 = ydata2 - y_prediction2
sns.barplot(x=xdata2[var2], y=residuals2, ax=ax, ci=None)
ax.set_box_aspect(1)
xbins = sorted(xdata2[var2].unique())
xticks= np.arange(0, len(xbins))
ax.set_xticks(xticks)
ax.set_xticklabels([round(xbins[i],1) if i in xticks[0::5] else '' \
                   for i in xticks])

pl.subplots_adjust(bottom=0.3, right=0.8, wspace=0.5)


In [ ]:
ydata = residuals2.copy()

xdata0 = data2[var1].copy()
xdata0 = xdata0[:, np.newaxis]
xdata = StandardScaler().fit_transform(xdata0) #xd = scaler.fit_transform(xdata)
xdata_scaled = pd.DataFrame(np.squeeze(xdata), columns=[x_var])
xdata_scaled.dropna().head()

# Split data
LR = LinearRegression().fit(xdata_scaled,ydata)
# predicting the accuracy score
y_prediction =  LR.predict(xdata_scaled)
R2=skmetrics.r2_score(ydata,y_prediction)
MSE = skmetrics.mean_squared_error(ydata,y_prediction)
print('r2 score: %.2f' % R2)
print('mean_sqrd_error: %.2f' % MSE)
print('RMSE: %.2f' % np.sqrt(MSE))
residuals = ydata.values - y_prediction
variance_resid = np.var(residuals)
variance_test = np.var(ydata)
variance_explained = 1-(variance_resid/variance_test)
print(variance_explained)

In [ ]:
data_ = vg.groupby([var1, var2, 'datakey']).median().reset_index()

fig, axn = pl.subplots(1, 2, figsize=(6,3))
ax=axn[0]
sns.regplot(x=var1, y='pearsons_size', data=data_, ax=ax,
            scatter_kws=scatter_kws)
ax=axn[1]
sns.regplot(x=var2, y='pearsons_size', data=data_, ax=ax,
            scatter_kws=scatter_kws)
pl.subplots_adjust(bottom=0.2, right=0.8)


In [ ]:
metric = 'pearsons_size'
ydata = data_[metric].copy()
xdata0 = data_[factors].copy()
#xdata0 = xdata0[:, np.newaxis]
xdata = StandardScaler().fit_transform(xdata0) #xd = scaler.fit_transform(xdata)
xdata_scaled = pd.DataFrame(np.squeeze(xdata), columns=[factors])
xdata_scaled.dropna().head()

# Split data
LR = LinearRegression().fit(xdata_scaled,ydata)
# predicting the accuracy score
y_prediction =  LR.predict(xdata_scaled)
R2=skmetrics.r2_score(ydata,y_prediction)
MSE = skmetrics.mean_squared_error(ydata,y_prediction)
print('r2 score: %.2f' % R2)
print('mean_sqrd_error: %.2f' % MSE)
print('RMSE: %.2f' % np.sqrt(MSE))
residuals = ydata.values - y_prediction
variance_resid = np.var(residuals)
variance_test = np.var(ydata)
variance_explained = 1-(variance_resid/variance_test)
print(variance_explained)